In [1]:
#cargar paquetes
import pandas as pd 
import sqlite3 as sql 
#import a_funciones as funciones  
import sys 
from sklearn.impute import SimpleImputer
import funciones as funciones#carga el archivo aparte con las funciones

In [2]:
#cargar bases de datos
df_employee_survey=pd.read_csv("https://raw.githubusercontent.com/vladimir-martin/analitica3_HR/d44db9d7c1014ad6b7a588f49d6505e4a02835cd/employee_survey_data.csv")
df_general=pd.read_csv("https://raw.githubusercontent.com/vladimir-martin/analitica3_HR/d44db9d7c1014ad6b7a588f49d6505e4a02835cd/general_data.csv",sep = ';')
df_manager=pd.read_csv("https://raw.githubusercontent.com/vladimir-martin/analitica3_HR/d44db9d7c1014ad6b7a588f49d6505e4a02835cd/manager_survey_data.csv")
df_retirement=pd.read_csv("https://raw.githubusercontent.com/vladimir-martin/analitica3_HR/d44db9d7c1014ad6b7a588f49d6505e4a02835cd/retirement_info.csv",sep = ';')
#df_out_time=pd.read_csv("https://github.com/vladimir-martin/analitica3_HR/raw/d44db9d7c1014ad6b7a588f49d6505e4a02835cd/out_time.csv")
#df_in_time=pd.read_csv("https://github.com/vladimir-martin/analitica3_HR/raw/d44db9d7c1014ad6b7a588f49d6505e4a02835cd/in_time.csv")



##Vista preliminar de las bases,nulos, categorias y outliners

pasos por tabla: 
1.nulos (en python) ok
2. analizar categorias (sql desde python) ok
3. poner formatos adecuados ( categorica, numerica,fecha etc)(python) ok
4. eliminar variables que no aportan( sql)ok
5. crear variable respuesta(atrition) (sql)ok
6. unir bases de datos (sql)

In [3]:
### llevar bases de datos a sql

conn= sql.connect("db_empleados.db") ### crea una base de datos con el nombre dentro de comillas, si existe crea una conexión.

### mostrar tablas dentro de la base de datos
cursor=conn.cursor() 
cursor.execute("select name from sqlite_master where type='table'")
cursor.fetchall()

[('employee_survey',),
 ('general',),
 ('manager',),
 ('retirement',),
 ('Attrition',),
 ('employee_survey_2',),
 ('general_2',),
 ('manager_2',),
 ('retirement_2',),
 ('base_full',)]

In [4]:
#tabla employe_survey
print(df_employee_survey.info())
df_employee_survey.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4410 non-null   int64  
 1   EnvironmentSatisfaction  4385 non-null   float64
 2   JobSatisfaction          4390 non-null   float64
 3   WorkLifeBalance          4372 non-null   float64
dtypes: float64(3), int64(1)
memory usage: 137.9 KB
None


,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,1,3.0,4.0,2.0
1,2,3.0,2.0,4.0
2,3,2.0,2.0,1.0
3,4,4.0,4.0,3.0
4,5,4.0,1.0,3.0


In [5]:
#TRATO DE NULOS CON MEDIANA
df_employee_survey =funciones.impute_columns(df_employee_survey, ['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance'], 'median')
df_employee_survey.isnull().sum()

EmployeeID                 0
EnvironmentSatisfaction    0
JobSatisfaction            0
WorkLifeBalance            0
dtype: int64

In [6]:
#asignacion correcta de tipo de variable
#las siguientes variables se coportan como categoricas, pasan ser categoricas
df_employee_survey=df_employee_survey.astype({"EnvironmentSatisfaction":int,"JobSatisfaction":int,"WorkLifeBalance":int})
df_employee_survey=df_employee_survey.astype({"EnvironmentSatisfaction":object,"JobSatisfaction":object,"WorkLifeBalance":object})
print(df_employee_survey.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   EmployeeID               4410 non-null   int64 
 1   EnvironmentSatisfaction  4410 non-null   object
 2   JobSatisfaction          4410 non-null   object
 3   WorkLifeBalance          4410 non-null   object
dtypes: int64(1), object(3)
memory usage: 137.9+ KB
None


In [7]:
### Llevar tablas a base de datos sql
df_employee_survey.to_sql("employee_survey",conn,if_exists="replace")

##ver categorias
#funciones.categorias(df_employee_survey,"employee_survey")
funciones.categorias(df_employee_survey,"employee_survey")


****Variable EmployeeID****
      EmployeeID  qty
0              1    1
1              2    1
2              3    1
3              4    1
4              5    1
...          ...  ...
4405        4406    1
4406        4407    1
4407        4408    1
4408        4409    1
4409        4410    1

[4410 rows x 2 columns]
****Variable EnvironmentSatisfaction****
   EnvironmentSatisfaction   qty
0                        1   845
1                        2   856
2                        3  1375
3                        4  1334
****Variable JobSatisfaction****
   JobSatisfaction   qty
0                1   860
1                2   840
2                3  1343
3                4  1367
****Variable WorkLifeBalance****
   WorkLifeBalance   qty
0                1   239
1                2  1019
2                3  2698
3                4   454


In [27]:
#tabla general data
print(df_general.info())
df_general.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   BusinessTravel           4410 non-null   object 
 2   Department               4410 non-null   object 
 3   DistanceFromHome         4410 non-null   int64  
 4   Education                4410 non-null   int64  
 5   EducationField           4410 non-null   object 
 6   EmployeeCount            4410 non-null   int64  
 7   EmployeeID               4410 non-null   int64  
 8   Gender                   4410 non-null   object 
 9   JobLevel                 4410 non-null   int64  
 10  JobRole                  4410 non-null   object 
 11  MaritalStatus            4410 non-null   object 
 12  MonthlyIncome            4410 non-null   int64  
 13  NumCompaniesWorked       4391 non-null   float64
 14  Over18                  

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,1,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,1,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,4,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,3,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,1,...,4.0,Y,12,8,2,9.0,2,6,0,4


In [28]:
#TRATO DE NULOS CON MEDIANA
df_general=funciones.impute_columns(df = df_general, columns = ['NumCompaniesWorked', 'TotalWorkingYears'], strategy = 'median')
df_general.isnull().sum()

Age                        0
BusinessTravel             0
Department                 0
DistanceFromHome           0
Education                  0
EducationField             0
EmployeeCount              0
EmployeeID                 0
Gender                     0
JobLevel                   0
JobRole                    0
MaritalStatus              0
MonthlyIncome              0
NumCompaniesWorked         0
Over18                     0
PercentSalaryHike          0
StandardHours              0
StockOptionLevel           0
TotalWorkingYears          0
TrainingTimesLastYear      0
YearsAtCompany             0
YearsSinceLastPromotion    0
YearsWithCurrManager       0
dtype: int64

In [29]:
######asignacion correcta de tipo de variable######
#las siguientes variables se coportan como categoricas, pasan ser categoricas
df_general=df_general.astype({"StockOptionLevel":object,"JobLevel":object,"Education":object,"TrainingTimesLastYear":object})
#se pasan variables a float para su manejo posterior en los modelos
df_general=df_general.astype({"Age":float,"DistanceFromHome":float,"EmployeeCount":float,"MonthlyIncome":float,"PercentSalaryHike":float,"StandardHours":float,"YearsAtCompany":float,"YearsSinceLastPromotion":float,"YearsWithCurrManager":float})
print(df_general.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   float64
 1   BusinessTravel           4410 non-null   object 
 2   Department               4410 non-null   object 
 3   DistanceFromHome         4410 non-null   float64
 4   Education                4410 non-null   object 
 5   EducationField           4410 non-null   object 
 6   EmployeeCount            4410 non-null   float64
 7   EmployeeID               4410 non-null   int64  
 8   Gender                   4410 non-null   object 
 9   JobLevel                 4410 non-null   object 
 10  JobRole                  4410 non-null   object 
 11  MaritalStatus            4410 non-null   object 
 12  MonthlyIncome            4410 non-null   float64
 13  NumCompaniesWorked       4410 non-null   float64
 14  Over18                  

In [30]:
### Llevar tablas a base de datos sql
df_general.to_sql("general",conn,if_exists="replace")
##ver categorias
funciones.categorias(df_general,"general")

****Variable Age****
     Age  qty
0   18.0   24
1   19.0   27
2   20.0   33
3   21.0   39
4   22.0   48
5   23.0   42
6   24.0   78
7   25.0   78
8   26.0  117
9   27.0  144
10  28.0  144
11  29.0  204
12  30.0  180
13  31.0  207
14  32.0  183
15  33.0  174
16  34.0  231
17  35.0  234
18  36.0  207
19  37.0  150
20  38.0  174
21  39.0  126
22  40.0  171
23  41.0  120
24  42.0  138
25  43.0   96
26  44.0   99
27  45.0  123
28  46.0   99
29  47.0   72
30  48.0   57
31  49.0   72
32  50.0   90
33  51.0   57
34  52.0   54
35  53.0   57
36  54.0   54
37  55.0   66
38  56.0   42
39  57.0   12
40  58.0   42
41  59.0   30
42  60.0   15
****Variable BusinessTravel****
      BusinessTravel   qty
0         Non-Travel   450
1  Travel_Frequently   831
2      Travel_Rarely  3129
****Variable Department****
               Department   qty
0         Human Resources   189
1  Research & Development  2883
2                   Sales  1338
****Variable DistanceFromHome****
    DistanceFromHome  qty
0      

In [31]:
#tabla manager survey
print(df_manager.info())
df_manager.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   EmployeeID         4410 non-null   int64
 1   JobInvolvement     4410 non-null   int64
 2   PerformanceRating  4410 non-null   int64
dtypes: int64(3)
memory usage: 103.5 KB
None


,EmployeeID,JobInvolvement,PerformanceRating
0,1,3,3
1,2,2,4
2,3,3,3
3,4,2,3
4,5,3,3


In [32]:
#asignacion correcta de tipo de variable
#las siguientes variables se coportan como categoricas, pasan ser categoricas
df_manager=df_manager.astype({"JobInvolvement":object,"PerformanceRating":object})
print(df_manager.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   EmployeeID         4410 non-null   int64 
 1   JobInvolvement     4410 non-null   object
 2   PerformanceRating  4410 non-null   object
dtypes: int64(1), object(2)
memory usage: 103.5+ KB
None


In [33]:
### Llevar tablas a base de datos sql
df_manager.to_sql("manager",conn,if_exists="replace")
##ver categorias
funciones.categorias(df_manager,"manager")

****Variable EmployeeID****
      EmployeeID  qty
0              1    1
1              2    1
2              3    1
3              4    1
4              5    1
...          ...  ...
4405        4406    1
4406        4407    1
4407        4408    1
4408        4409    1
4409        4410    1

[4410 rows x 2 columns]
****Variable JobInvolvement****
   JobInvolvement   qty
0               1   249
1               2  1125
2               3  2604
3               4   432
****Variable PerformanceRating****
   PerformanceRating   qty
0                  3  3732
1                  4   678


In [34]:
#tabla retirement info
print(df_retirement.info())
df_retirement.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   EmployeeID         711 non-null    int64 
 1   Attrition          711 non-null    object
 2   retirementDate     711 non-null    object
 3   retirementType     711 non-null    object
 4   resignationReason  641 non-null    object
dtypes: int64(1), object(4)
memory usage: 27.9+ KB
None


,EmployeeID,Attrition,retirementDate,retirementType,resignationReason
0,2,Yes,15/10/2016,Resignation,Others
1,7,Yes,20/07/2016,Resignation,Stress
2,14,Yes,08/11/2016,Resignation,Others
3,29,Yes,15/10/2016,Resignation,Others
4,31,Yes,08/02/2016,Resignation,Salary


In [35]:
#asignacion correcta de tipo de variable
#las siguientes variables pasa a ser tipo fecha
df_retirement["retirementDate"]=pd.to_datetime(df_retirement['retirementDate'])
print(df_retirement.info())
df_retirement.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   EmployeeID         711 non-null    int64         
 1   Attrition          711 non-null    object        
 2   retirementDate     711 non-null    datetime64[ns]
 3   retirementType     711 non-null    object        
 4   resignationReason  641 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 27.9+ KB
None


,EmployeeID,Attrition,retirementDate,retirementType,resignationReason
0,2,Yes,2016-10-15,Resignation,Others
1,7,Yes,2016-07-20,Resignation,Stress
2,14,Yes,2016-08-11,Resignation,Others
3,29,Yes,2016-10-15,Resignation,Others
4,31,Yes,2016-08-02,Resignation,Salary


In [36]:
### Llevar tablas a base de datos sql
df_retirement.to_sql("retirement",conn,if_exists="replace")

##ver categorias
funciones.categorias(df_retirement,"retirement")


****Variable EmployeeID****
     EmployeeID  qty
0             2    1
1             7    1
2            14    1
3            29    1
4            31    1
..          ...  ...
706        4382    1
707        4387    1
708        4389    1
709        4392    1
710        4403    1

[711 rows x 2 columns]
****Variable Attrition****
  Attrition  qty
0       Yes  711
****Variable retirementDate****
          retirementDate  qty
0    2016-01-02 00:00:00    2
1    2016-01-03 00:00:00    2
2    2016-01-04 00:00:00    1
3    2016-01-05 00:00:00    4
4    2016-01-06 00:00:00    6
..                   ...  ...
303  2016-12-12 00:00:00    2
304  2016-12-13 00:00:00    4
305  2016-12-14 00:00:00    3
306  2016-12-15 00:00:00    3
307  2016-12-16 00:00:00    1

[308 rows x 2 columns]
****Variable retirementType****
  retirementType  qty
0          Fired   70
1    Resignation  641
****Variable resignationReason****
  resignationReason  qty
0              None   70
1            Others  323
2          

In [37]:
#analisis con año de retiro 
pd.read_sql("""select strftime('%Y',retirementDate) as fecha, 
                                count(*) as qty
                                from retirement 
                                group by fecha""", conn)


,fecha,qty
0,2016,711


In [38]:
#analisis del mes de retiro
pd.read_sql("""select strftime('%m',retirementDate) as Mes, 
                                count(*) as qty
                                from retirement 
                                group by Mes""", conn)

,Mes,qty
0,01,72
1,02,61
2,03,62
3,04,55
4,05,69
5,06,63
6,07,77
7,08,59
8,09,46
9,10,61


se crea archivo preprocesamiento.sql para hacer la eliminacion de variables que no aportan, de acuerdo al analisis de categorias y para unir las bases.

In [47]:
#### para hacer todos los preprocesamienteos se crea archivo .sql que se ejecuta con la función: ejecutar_sql del archivo funciones.py
cur=conn.cursor()
funciones.ejecutar_sql('preprocesamiento.sql',cur)
df_completo=pd.read_sql("select * from base_full  ",conn)
print("****Numero de registros de la tabla*** \n  ", pd.read_sql("select count(distinct EmployeeID)  from base_full  ",conn))
print(df_completo.info())
df_completo.describe(include='all')

****Numero de registros de la tabla*** 
      count(distinct EmployeeID)
0                        4410
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4410 non-null   int64  
 1   Age                      4410 non-null   float64
 2   BusinessTravel           4410 non-null   object 
 3   Department               4410 non-null   object 
 4   DistanceFromHome         4410 non-null   float64
 5   Education                4410 non-null   int64  
 6   EducationField           4410 non-null   object 
 7   Gender                   4410 non-null   object 
 8   JobLevel                 4410 non-null   int64  
 9   JobRole                  4410 non-null   object 
 10  MaritalStatus            4410 non-null   object 
 11  MonthlyIncome            4410 non-null   float64
 12  NumCompaniesWorked       4410

,EmployeeID,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,...,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
count,4410.000000,4410.000000,4410,4410,4410.000000,4410.000000,4410,4410,4410.000000,4410,...,4410.000000,4410.000000,4410.000000,4410.000000,4410,4410.000000,4410.000000,4410.000000,4410.000000,4410.000000
unique,NaN,NaN,3,3,NaN,NaN,6,2,NaN,9,...,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,Travel_Rarely,Research & Development,NaN,NaN,Life Sciences,Male,NaN,Sales Executive,...,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,3129,2883,NaN,NaN,1818,2646,NaN,978,...,NaN,NaN,NaN,NaN,3699,NaN,NaN,NaN,NaN,NaN
mean,2205.500000,36.923810,NaN,NaN,9.192517,2.912925,NaN,NaN,2.063946,NaN,...,2.799320,7.008163,2.187755,4.123129,NaN,2.729932,3.153741,2.725170,2.729478,2.763492
std,1273.201673,9.133301,NaN,NaN,8.105026,1.023933,NaN,NaN,1.106689,NaN,...,1.288978,6.125135,3.221699,3.567327,NaN,0.711400,0.360742,1.089852,1.098904,0.703541
min,1.000000,18.000000,NaN,NaN,1.000000,1.000000,NaN,NaN,1.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,NaN,1.000000,3.000000,1.000000,1.000000,1.000000
25%,1103.250000,30.000000,NaN,NaN,2.000000,2.000000,NaN,NaN,1.000000,NaN,...,2.000000,3.000000,0.000000,2.000000,NaN,2.000000,3.000000,2.000000,2.000000,2.000000
50%,2205.500000,36.000000,NaN,NaN,7.000000,3.000000,NaN,NaN,2.000000,NaN,...,3.000000,5.000000,1.000000,3.000000,NaN,3.000000,3.000000,3.000000,3.000000,3.000000
75%,3307.750000,43.000000,NaN,NaN,14.000000,4.000000,NaN,NaN,3.000000,NaN,...,3.000000,9.000000,3.000000,7.000000,NaN,3.000000,3.000000,4.000000,4.000000,3.000000


In [40]:
#se quedo con un dataframe de 4410 registros, 26 variables sin codificar(generad dummies), de las variables categoricas.
df_completo.shape

(4410, 26)